This project is to find each preoperty information.
1. if the closest station has direct line to Melbourne Central during weekday from 7 a.m. to 9 a.m. or not. 
2. How long would it take (average).
3. number of houses in each suburb.
4. number of units in each suburb (condo/apartment)
5. municipality of the property suburb(LGA).
6. Percentage of the Australian-born population in the property suburb.
7. median income of the population in the property suburb 
8. population in the property suburb 


In [1]:
# for all library
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point # use for plot in geometry
from pdfminer.high_level import extract_text # for extract pdf text from pdfminer.six
import re # for regex
import ast # for convert to appropriate type
import geopy.distance # for distance calculation - property to station



# Extract data from files

In [2]:
import pandas as pd 
jpath = 'data/house1.json' # path for json file
xpath = 'data/house2.xml' # path for xml file

## JSON DATA

In [3]:
try:
    j = pd.read_json(jpath) # use read json to parse json file
except Exception as e:
    print(e)

In [4]:
j.head(2)

,property_id,lat,lng,addr_street
0,29654,-37.783257,145.000198,2 Short Street
1,84314,-37.851406,145.003433,7 Pridham Street


## XML DATA

In [5]:
try:
    x = pd.read_xml('xpath')
except Exception as e:
    print(e)

Start tag expected, '<' not found, line 1, column 1 (<string>, line 1)


### Fix XML File

Check XML file

In [6]:
with open(xpath,'r') as file:
    xfile = file.read()  # all xml
print(xfile[0:300],
xfile[-100:]) 

<property>
  <property_id>53857</property_id>
  <lat>-37.83034134</lat>
  <lng>144.9947815</lng>
  <addr_street>86 Balmain Street</addr_street>
</property>
<property>
  <property_id>68409</property_id>
  <lat>-37.856339</lat>
  <lng>145.02240700000004</lng>
  <addr_street>25 Clarendon Street</addr_s t>-37.741972</lat>
  <lng>145.018771</lng>
  <addr_street>126 Gower Street</addr_street>
</property>


It did not have wrapper 

In [7]:
x = "<property_list>" + xfile + "</property_list>"  # add wrapper

In [8]:
print(x[0:200],
x[-200:])

<property_list><property>
  <property_id>53857</property_id>
  <lat>-37.83034134</lat>
  <lng>144.9947815</lng>
  <addr_street>86 Balmain Street</addr_street>
</property>
<property>
  <property_id>684 Street</addr_street>
</property>
<property>
  <property_id>31804</property_id>
  <lat>-37.741972</lat>
  <lng>145.018771</lng>
  <addr_street>126 Gower Street</addr_street>
</property></property_list>


In [9]:
try:
    x = pd.read_xml(x)
except Exception as e:
    print(e)

xmlParseEntityRef: no name, line 3107, column 18 (<string>, line 3107)



Still have syntax error: no name, line 3107, column 18.
So check line 3107 in the initial file

In [10]:
# create list to access each line.
with open(xpath, "r") as xl: # open the file again
    alllines = xl.readlines() # use realines to make each line into element of list
# access line 3107 as index 3106
alllines[3106]

'  <addr_street>2&3/17-19 Browning Street</addr_street>\n'

***<** and **&** are strictly illegal in XML need to change to pre-defined entity*

There are 5 pre-defined entity references in XML:
|XML|Sign|Def|
|----|----|---------|
|\&lt;|<|less than|
|\&gt;|>|greater than|
|\&amp;|&|ampersand|
|\&apos;|'|apostrophe|
|\&quot;|"|quotation mark|

ref : https://www.w3schools.com/xml/xml_syntax.asp

In [11]:
x = x.replace("&","&amp;") # adjust to be XML sign

In [12]:
try:
    x = pd.read_xml(x)
except Exception as e:
    print(e)

In [13]:
x.head(2)

,property_id,lat,lng,addr_street
0,53857,-37.830341,144.994781,86 Balmain Street
1,68409,-37.856339,145.022407,25 Clarendon Street


## Combine JSON and XML df

In [14]:
x.shape,j.shape

((1015, 4), (1019, 4))

In [15]:
property = pd.concat([x,j], ignore_index=True) # combine file, index need to be re-running so ignore
property.shape

(2034, 4)

# Cleaning the combined file

## check and manage duplicates

There is key column, property_id, which each property should have distince value. Therefore anchor in that column if have any duplicate, drop it out.

In [16]:
unique_property = property.drop_duplicates(subset='property_id').copy() #copy from original list

In [17]:
unique_property = unique_property.reset_index(drop=True) #reset index

In [18]:
unique_property.shape # remove 22 rowa

(2012, 4)

In [19]:
unique_property.head(2)

,property_id,lat,lng,addr_street
0,53857,-37.830341,144.994781,86 Balmain Street
1,68409,-37.856339,145.022407,25 Clarendon Street


In [20]:
(unique_property== 'NA').sum() # check NA

property_id    0
lat            0
lng            0
addr_street    0
dtype: int64

# Find suburb of each property

## Clean suburb file

In [21]:
import geopandas as gpd

In [22]:
sub_path = "supplementary_data/Vic_suburb_boundary/VIC_LOCALITY_POLYGON_shp.shp"

try:
    sub = gpd.read_file(sub_path)
except Exception as e:
    print(e)

In [23]:
sub.head(2)

,LC_PLY_PID,DT_CREATE,DT_RETIRE,LOC_PID,VIC_LOCALI,VIC_LOCA_1,VIC_LOCA_2,VIC_LOCA_3,VIC_LOCA_4,VIC_LOCA_5,VIC_LOCA_6,VIC_LOCA_7,geometry
0,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"POLYGON ((141.74552 -35.07229, 141.74552 -35.0..."
1,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"POLYGON ((148.66877 -37.39571, 148.66876 -37.3..."


In [24]:
subclean = sub[['VIC_LOCA_2','geometry']] # select only needed column
subclean.rename(columns={'VIC_LOCA_2':'suburb'}, inplace=True) # change column name
subclean.head(2)

/var/folders/hn/mgkfr3qd4652k_9wn776wn4m0000gn/T/ipykernel_10567/1973556953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subclean.rename(columns={'VIC_LOCA_2':'suburb'}, inplace=True) # change column name


,suburb,geometry
0,UNDERBOOL,"POLYGON ((141.74552 -35.07229, 141.74552 -35.0..."
1,NURRAN,"POLYGON ((148.66877 -37.39571, 148.66876 -37.3..."


## Find suburb of each property

Need to find suburb as the stations

In [25]:
from shapely.geometry import Point # use for plot in geometry

In [26]:
def findsuburbname(long,lat):
    '''
    use the location of property to find suburb
    '''
    
    proplocation = Point(long,lat)
    # locate where True(geo contain proplocation) then get suburb
    try:
        suburb = subclean.loc[subclean['geometry'].contains(proplocation), 'suburb'].values[0] 
    except:
        suburb = "NA"
    
    return suburb

In [27]:
unique_property['suburb'] = unique_property.apply(lambda x: findsuburbname(x['lng'],x['lat']), axis =1)
unique_property.head(2)

,property_id,lat,lng,addr_street,suburb
0,53857,-37.830341,144.994781,86 Balmain Street,CREMORNE
1,68409,-37.856339,145.022407,25 Clarendon Street,ARMADALE


In [28]:
unique_property.shape # recheck

(2012, 5)

# Fill LGA

## Extract LGA from pdf

In [29]:
from pdfminer.high_level import extract_text # for extract pdf text from pdfminer.six

lga_path = "supplementary_data/Lga_to_suburb.pdf"
lga = extract_text(lga_path)
lga[:200]

"BANYULE : ['Abbotsford', 'Airport West', 'Albert Park', 'Alphington'] \n\nBRIMBANK : ['Altona', 'Altona North', 'Armadale', 'Ascot Vale'] \n\nDAREBIN : ['Ashburton', 'Ashwood', 'Avondale Heights', 'Balacl"

Extract each LGA and its suburbs with RegEX

In [30]:
import re # for regex
lgasub_patt =r'[A-Z ]+ ?: ?\[[^\]]+\]'  # From first character to closing bracket

In [31]:
lgaandsub = re.findall(lgasub_patt, lga) 
lgaandsub[0:3]

["BANYULE : ['Abbotsford', 'Airport West', 'Albert Park', 'Alphington']",
 "BRIMBANK : ['Altona', 'Altona North', 'Armadale', 'Ascot Vale']",
 "DAREBIN : ['Ashburton', 'Ashwood', 'Avondale Heights', 'Balaclava']"]

The information is in string - convert to dict to work with easier

In [32]:
import ast # for convert to appropriate type

def stringtodict(lgasub):

    '''
    To convert string of LGA and subs to dict, identify key and  value 
    as the string is not in actual form of dict (no prenthesis) or list (has colon)
    '''
    k,v = lgasub.split(":",1) 
    key = k.strip()
    value = ast.literal_eval(v.strip())

    return {key:value}

In [33]:
lga_sub = {key:value for x in lgaandsub for key,value in stringtodict(x).items()}
lga_sub["BANYULE"]

['Abbotsford', 'Airport West', 'Albert Park', 'Alphington']

In [34]:
def findlga(suburb):
    """
    if suburb is match, return lga
    """
    for k,v in lga_sub.items(): # in each set of suburbs of LGA
        for i in v: # in each suburb in the LGA
            if suburb == i.upper(): # if suburb matched with suburb in the set
                 return k.upper() # lga is k
    return "NA"

In [35]:
findlga('ARMADALE') # test

'BRIMBANK'

In [36]:
unique_property['LGA'] = unique_property['suburb'].apply(findlga)
unique_property.head(2)

,property_id,lat,lng,addr_street,suburb,LGA
0,53857,-37.830341,144.994781,86 Balmain Street,CREMORNE,WELLINGTON
1,68409,-37.856339,145.022407,25 Clarendon Street,ARMADALE,BRIMBANK


# Find closest station

In [37]:
stoppath = "supplementary_data/Vic_GTFS_data/metropolitan/stops.txt"
with open(stoppath,"r") as stops:
    s = stops.readlines()

s[0:3]

['\ufeffstop_id,stop_name,stop_short_name,stop_lat,stop_lon\n',
 '"15351","Sunbury Railway Station","Sunbury","-37.5790909705787","144.727318781329"\n',
 '"15353","Diggers Rest Railway Station","Diggers Rest","-37.6270165149357","144.719922441113"\n']

In [38]:
allstop = s[1:] # remove header
allstop[0:3]

['"15351","Sunbury Railway Station","Sunbury","-37.5790909705787","144.727318781329"\n',
 '"15353","Diggers Rest Railway Station","Diggers Rest","-37.6270165149357","144.719922441113"\n',
 '"19827","Stony Point Railway Station","Crib Point","-38.3742345364937","145.221837462187"\n']

## Clean station file

In [39]:
allstop_clean = [x.replace("\n",'') for x in allstop] # remove newling symbol
allstop_clean[0]

'"15351","Sunbury Railway Station","Sunbury","-37.5790909705787","144.727318781329"'

In [40]:
# make each value convert to suitable data type and contain in a list
stopsub = [list(ast.literal_eval(x)) for x in allstop_clean]
stopsub[0:2]

[['15351',
  'Sunbury Railway Station',
  'Sunbury',
  '-37.5790909705787',
  '144.727318781329'],
 ['15353',
  'Diggers Rest Railway Station',
  'Diggers Rest',
  '-37.6270165149357',
  '144.719922441113']]

In [41]:
# create nested for easier access
stops_dict = {int(i[0]): {'station_name': i[1], 
                     'suburb': i[2], 
                     'lat': float(i[3]), 
                     'lng': float(i[4])} for i in stopsub}

In [42]:
stops_dict[15351] # test

{'station_name': 'Sunbury Railway Station',
 'suburb': 'Sunbury',
 'lat': -37.5790909705787,
 'lng': 144.727318781329}

## Finding closest station to each property using Geopy

### Calculate distance from property to station

In [43]:
import geopy.distance

def caldistance(plat,plng,slat,slng):
    '''
     Calculate distance by using property's latitude, longitude 
    and station's latitude, longitude using geopy.

    ref: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
    '''

    coords_1 = (plat, plng)
    coords_2 = (slat, slng)

    return geopy.distance.geodesic(coords_1, coords_2).km

In [44]:
def shotestdistance(plat,plng):
    """
    finding the shortest distance from property to station
    to get the closest station info
    """
    currdis = [] # initiate current distance
    for k,v in stops_dict.items(): # each station in the stopdict
        slat = v['lat']
        slng = v['lng']
        
        distance = caldistance(plat,plng,slat,slng) # calculate distance

        if len(currdis) == 0: # for the first stop 
            currdis.append(distance) # add distance
            station_id = k
            station_distance = round(currdis[0],2)

        elif len(currdis) > 0: # for other stops
            if distance < currdis[0]: # if distance less than previous shortest distance
                currdis[0] = distance # replace the distance in currdis
                station_id = k
                station_distance = round(currdis[0],2)
                
            else: # if distance is greater, skip it
                pass
        
    return station_id,station_distance

In [45]:
unique_property.head(2)

,property_id,lat,lng,addr_street,suburb,LGA
0,53857,-37.830341,144.994781,86 Balmain Street,CREMORNE,WELLINGTON
1,68409,-37.856339,145.022407,25 Clarendon Street,ARMADALE,BRIMBANK


In [46]:
shotestdistance(-37.830341,144.994781) # test1

(19907, 0.48)

In [47]:
shotestdistance(-37.848385,144.870640) # test2

(19994, 1.3)

### Apply function to get station id

In [48]:
unique_property[['station_id','station_distance']] = unique_property.apply(lambda x: shotestdistance(x['lat'],x['lng']),
                                                                           axis = 1,
                                                                           result_type='expand')

In [49]:
unique_property[0:10]


,property_id,lat,lng,addr_street,suburb,LGA,station_id,station_distance
0,53857,-37.830341,144.994781,86 Balmain Street,CREMORNE,WELLINGTON,19907.0,0.48
1,68409,-37.856339,145.022407,25 Clarendon Street,ARMADALE,BRIMBANK,19945.0,0.27
2,4719,-37.848385,144.870640,8A Berty Street,NEWPORT,MANSFIELD,19994.0,1.30
3,59357,-37.808037,145.185867,12 Mountfield Road,MITCHAM,PORT PHILLIP,19900.0,1.23
4,92720,-38.007039,145.084811,14 Montgomery Street,MORDIALLOC,PORT PHILLIP,19863.0,0.26
5,65607,-37.779045,145.314018,10 Anaba Court,MOOROOLBARK,WEST WIMMERA,19877.0,0.65
6,37388,-37.642063,145.034897,4 Bristol Court,EPPING,FRANKSTON,20005.0,1.22
7,79301,-37.954121,145.143189,67 Regent Ave,SPRINGVALE,GREATER BENDIGO,19913.0,1.04
8,65279,-37.807124,145.302976,29 Landstrom Quadrant,KILSYTH,SURF COAST,19878.0,2.36
9,12665,-37.717783,144.768230,21 Cottrell Court,DELAHEY,BULOKE,20000.0,1.92


# Find Travel Time

In [50]:
calendarpath = "supplementary_data/Vic_GTFS_data/metropolitan/calendar.txt"
with open(calendarpath, 'r') as c:
    calendar = c.readlines()
calendar[0:3]

['\ufeffservice_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date\n',
 '"T2","0","0","0","0","0","1","0","20151009","20151011"\n',
 '"UJ","0","0","0","0","0","0","1","20151009","20151011"\n']

In [51]:
# remove header and new line \n
calendar_clean = calendar[1:]
calendar_clean = [x.replace('\n','') for x in calendar_clean]
calendar_clean[0:3]

['"T2","0","0","0","0","0","1","0","20151009","20151011"',
 '"UJ","0","0","0","0","0","0","1","20151009","20151011"',
 '"T6","0","0","0","0","1","0","0","20151009","20151011"']

In [52]:
# make it proper data type
calendar_clean = [list(ast.literal_eval(x)) for x in calendar_clean]
calendar_clean[0:3]

[['T2', '0', '0', '0', '0', '0', '1', '0', '20151009', '20151011'],
 ['UJ', '0', '0', '0', '0', '0', '0', '1', '20151009', '20151011'],
 ['T6', '0', '0', '0', '0', '1', '0', '0', '20151009', '20151011']]

### Find service_code of train that running on any week day which is index 1 to 5 (inclusive)

In [62]:
runonweekday = [x[0] for x in calendar_clean if '1' in x[1:6]]
runonweekday

['T6',
 'T5',
 'T6_1',
 'T5_1',
 'T0',
 'T0+a6',
 'T0+a5',
 'T5+tg',
 'T5+ph',
 'T5+tg_1',
 'T5+ph_1',
 'T5+ao',
 'T5+ta']

### Look at the stop time file to see if the train direct to Melbourne Central during 7 - 9 am

In [63]:
stoptime_path = "supplementary_data/Vic_GTFS_data/metropolitan/stop_times.txt"
with open(stoptime_path, 'r') as s:
    sttimes = s.readlines()

sttimes[0:3]

['\ufefftrip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled\n',
 '"17182517.T2.2-ALM-B-mjp-1.1.H","04:57:00","04:57:00","19847","1","","0","0","0"\n',
 '"17182517.T2.2-ALM-B-mjp-1.1.H","04:58:00","04:58:00","19848","2","","0","0","723.017818057993"\n']

In [64]:
len(sttimes)

390306

In [65]:
# remove header and new line \n
sttimes_clean = sttimes[1:]
sttimes_clean = [x.replace('\n','') for x in sttimes_clean]
sttimes_clean[0:3]

['"17182517.T2.2-ALM-B-mjp-1.1.H","04:57:00","04:57:00","19847","1","","0","0","0"',
 '"17182517.T2.2-ALM-B-mjp-1.1.H","04:58:00","04:58:00","19848","2","","0","0","723.017818057993"',
 '"17182517.T2.2-ALM-B-mjp-1.1.H","05:00:00","05:00:00","19849","3","","0","0","1951.73507163902"']